In [3]:
from functions import *
from torch.nn.utils import clip_grad_value_
from torch.optim import AdamW,Adam,SGD
LOCAL = True
if LOCAL:
    dataPath = '/home/zhenlan/Desktop/Projects/HMS/Data/'

In [9]:
# HP
config = {
    "d_model": 768,
    "n_layer": 24,
    "rms_norm": True,
    "normOverChannel": True,
    "in_channels": 20, # if EEG
    "seqLen": 10000,
    "dataFolder": "train_eegs_LocalNorm",
    "classWeight": 1.0
}

config.update(fixed_config)
config = Config(config)

# other training parameters
epochs = 6
batch_size = 2
accumulation_steps = 8
lr = 1e-4
clip = 1e-2

In [5]:
# Data
train = pd.read_pickle(dataPath+'train_data.pkl')
val = pd.read_pickle(dataPath+'val_data.pkl')
trainData = eegData(train,dataPath,config.dataFolder)
valData = eegData(val,dataPath,config.dataFolder)

train_loader = DataLoader(trainData, batch_size=batch_size, shuffle=True, num_workers=16)
val_loader = DataLoader(valData, batch_size=batch_size, shuffle=False, num_workers=16)

In [5]:
model = seq2seqModel(config).to('cuda')
trainable_params = model.parameters()
optimizer = AdamW(trainable_params,lr = lr,amsgrad=True)
#opt = Adam(paras,lr = lr)
# opt = SGD(paras,lr = lr)

In [ ]:
for epoch in range(epochs):
    model.train()
    train_loss = np.zeros(2)
    skip = 0
    # train
    for i, (input_ids,targets) in enumerate(train_loader):
        input_ids,targets = input_ids.to('cuda'),targets.to('cuda')
        autoL,classL = model(input_ids,targets)
        loss = autoL + config.classWeight * classL
        loss.backward()
        train_loss += np.array([autoL.detach().cpu(),classL.detach().cpu()])
        # print(i,train_loss)
        if (i + 1) % accumulation_steps == 0:
            clip_grad_value_(trainable_params,clip)
            optimizer.step()
            optimizer.zero_grad()
    train_loss /= (i+1)

    # eval
    eval_loss = np.zeros(2)
    model.eval()
    with torch.no_grad():
        for i, (input_ids,targets) in enumerate(val_loader):
                input_ids,targets = input_ids.to('cuda'),targets.to('cuda')
                autoL,classL = model(input_ids,targets)
                eval_loss += np.array([autoL.detach().cpu(),classL.detach().cpu()])
    eval_loss /= (i+1)
    print(f"epoch {epoch}: train autoL {train_loss[0]} train classL {train_loss[1]}, eval autoL {eval_loss[0]} train classL {eval_loss[1]}")
    model.train()